In [24]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj
import sys
sys.path.append('../src')
import numpy as np
import seaborn as sns
import torch
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
plt.style.use('dark_background')
import data
from skorch.callbacks import Checkpoint
from skorch import NeuralNetRegressor
#import models
from tqdm import tqdm
import train_reg
import config
import pandas as pd
import features
#df = data.get_tracks() # note: different Xs can be different shapes
#df = data.remove_invalid_tracks(df)
#df = data.preprocess(df)
#df = data.add_outcome(df)
#n = df.shape[0]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
dset = 'clath_aux_dynamin'
outcome_def = 'Z_sig_mean'
out_dir = 'results/regression/Sep15'

In [87]:
feat_names = ['mean_square_displacement',
              'mean_total_displacement',
              'X_max',
              'X_mean',
              'X_std',
              'lifetime'
             ]

### Max strength prediction

In [91]:
train_reg.load_and_train(dset, outcome_def, out_dir, feat_names=feat_names)

/accounts/grad/xsli/.local/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
../src/train_reg.py:81: RuntimeWarning: invalid value encountered in less
  sigs = np.array(r[f'{track}_pvals']) < 0.05
100%|██████████| 5/5 [16:28<00:00, 182.89s/it]


In [14]:
df = train_reg.add_sig_mean(df)

../src/train_reg.py:77: RuntimeWarning: invalid value encountered in less
  sig_mean.append(0)


In [27]:
df_train = df[df.cell_num.isin(config.DSETS[dset]['train'])] 
df_test = df[df.cell_num.isin(config.DSETS[dset]['test'])] 

In [94]:
results = train_reg.load_results(out_dir)
r = results
r = r[[k for k in r if not 'std' in k]]
r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
r['r2']

model_type
svm       0.148
rf        0.196
linear    0.200
gb        0.225
ridge     0.200
Name: r2, dtype: float64

In [97]:
m = pd.read_pickle(f'{out_dir}/gb.pkl')

In [103]:
df_test = df_test.dropna()
test_preds = train_reg.test_reg(df_test, m['model'], outcome_def=outcome_def)

In [104]:
test_preds

{'preds': array([411.21317406, 516.00478196, 366.25512058, ..., 383.93901524,
        799.78401161, 466.74874401]),
 'r2': 0.2776157617689128,
 'pearsonr': (0.5305645133591516, 0.0)}